In [ ]:
import duckdb
from dotenv import find_dotenv, load_dotenv
import os
import traceback

load_dotenv(find_dotenv())

In [ ]:
if os.getenv("SA_PAT") is None:
    raise ValueError("SA_PAT is not set")

if os.getenv("SA_ROLE") is None:
    raise ValueError("SA_ROLE is not set")

if os.getenv("SNOWFLAKE_ACCOUNT_URL") is None:
    raise ValueError("SNOWFLAKE_ACCOUNT_URL is not set")

if os.getenv("DEMO_DATABASE") is None:
    raise ValueError("DEMO_DATABASE is not set")

pat_token = os.getenv("SA_PAT")
snowflake_account_url = os.getenv("SNOWFLAKE_ACCOUNT_URL")
catalog_uri = f"{snowflake_account_url}/polaris/api/catalog".lower()
role = os.getenv("SA_ROLE")
database = os.getenv("DEMO_DATABASE")

In [ ]:
secret_sql = f"""
  CREATE SECRET iceberg_pat_secret (
  TYPE iceberg,
  CLIENT_ID '',
  CLIENT_SECRET '{pat_token}',
  OAUTH2_SERVER_URI '{catalog_uri}/v1/oauth/tokens',
  OAUTH2_GRANT_TYPE 'client_credentials',
  OAUTH2_SCOPE 'session:role:{role}'
  );
"""

attach_sql = f"""
  ATTACH '{database}' AS {database} (
    TYPE iceberg,
    SECRET iceberg_pat_secret,
    ENDPOINT '{catalog_uri}',
    SUPPORT_NESTED_NAMESPACES false
  );
"""

#print("Secret SQL:")
#print(secret_sql)
#print("\nAttach SQL:")
# print(attach_sql)

In [ ]:
conn = duckdb.connect()
conn.execute("INSTALL iceberg;")
conn.execute("LOAD iceberg;")
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")
try:
    conn.execute(secret_sql)
    conn.execute(attach_sql)
except Exception as e:
    traceback.print_exc()


In [ ]:
try:
    tables_result = conn.execute("SHOW ALL TABLES").fetchall()
    first_table = tables_result[0]

    database, schema, name, column_names, column_types, temporary = first_table

    full_table_name = f"{database}.{schema}.{name}"

    print(f"Querying table: {full_table_name}")

    sample_query = f"SELECT * FROM {full_table_name} LIMIT 3;"

    sample_result = conn.execute(sample_query).fetchall()

    for i, row in enumerate(sample_result): # Show first 3 rows
        print(f" Row {i+1}: {row}")

except Exception as e:
    traceback.print_exc()